In [190]:
import os
import bs4
import numpy as np
from langchain_anthropic import ChatAnthropic
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import tiktoken
from langchain.load import loads, dumps
from operator import itemgetter
load_dotenv()
langchain_api = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

In [138]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(parse_only =bs4.SoupStrainer( class_=("post-content", "post-title", "post-header" )))
)

In [139]:
docs= loader.load()

In [66]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [67]:
out = vectorstore._collection.get(include=["documents", "metadatas"])
print(out.keys())

for i in range(min(5,len(out['ids']))):
    print("ID:", out["ids"][i])
    print("Text:", out["documents"][i][:200])
    print("Meta:", out["metadatas"][i])
    print("="*50)

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])
ID: beef1b17-0aae-4402-849d-f624debc1eee
Text: LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool con
Meta: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
ID: 33227605-ec9c-4783-b6f9-8a6f7199f8d5
Text: Memory

Short-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.
Long-term memory: This provides the agent with th
Meta: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
ID: b79bf339-9e4e-452a-aa5a-49c286d7c7a9
Text: Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a s
Meta: {'source': 'https

In [68]:
vectorstore._collection.count()

189

In [74]:
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_template(
    "Use the context to answer the question.\n\nContext:\n{context}\n\nQuestion: {question}"
)
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the context to answer the question.\n\nContext:\n{context}\n\nQuestion: {question}'), additional_kwargs={})]


In [75]:
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo',temperature=0)
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complex task into smaller and simpler steps to make it more manageable and easier to accomplish. This can be done through techniques like Chain of Thought (CoT) or Tree of Thoughts, which help in organizing and structuring the steps needed to complete a task.'

In [76]:
question = "Do you think Amir is going to become very rich by the age 40 years old?"
document = 'Amir will is going to gain networth of 200 million by the time he is 40 years ols. His income will get to at least 1 million per month.'

In [77]:
encoding = tiktoken.get_encoding("cl100k_base")
len(encoding.encode(question))

18

In [ ]:
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question) 
document_result = embd.embed_documents(document)

In [79]:
def cosine(vec1,vec2):
    dot_product = np.dot(vec1,vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

In [84]:
cosine(query_result,document_result)

np.float64(0.9074268061127665)

In [93]:
prompt = """ Answer the question based on the following contex:
{context}

Question: {question}

"""
prompt = ChatPromptTemplate.from_template(prompt)
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)


In [96]:
chain = prompt | llm | StrOutputParser()
chain.invoke({'context':docs, 'question':'What is Task Decomposition'})

'Task decomposition in the context of LLM-powered autonomous agents refers to breaking down large tasks into smaller, more manageable subgoals. This process enables the agent to efficiently handle complex tasks by dividing them into simpler steps. Task decomposition can be achieved through various methods, such as using simple prompting for LLM, task-specific instructions, or human inputs. Additionally, techniques like Chain of Thought and Tree of Thoughts are used to enhance model performance on complex tasks by decomposing them into multiple manageable steps.'

In [97]:
rag_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke('What is Task Decomposition')

"Task decomposition is a technique used by agents to break down complex tasks into smaller and simpler steps. This process involves transforming big tasks into multiple manageable tasks, allowing the agent to plan ahead and interpret the model's thinking process. Techniques like Chain of Thought (CoT) and Tree of Thoughts extend task decomposition by exploring multiple reasoning possibilities at each step and generating multiple thoughts per step, creating a tree structure."

Multi Query

In [187]:
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt = ChatPromptTemplate.from_template(template)
generate_queries = (
      prompt
    | ChatOpenAI(model='gpt-3.5-turbo',temperature=0)
    | StrOutputParser()
    | (lambda x:x.split('\n'))
)

def get_unique_union(documents):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]
    

# Testing

# queries = generate_queries.invoke("what is the Decomposition in LLM")
# batches = [retriever.get_relevant_documents(q) for q in queries]
# unique_docs = get_unique_union(batches)

question = "What is task decomposition for LLM agents?"
retriever_chain = generate_queries| retriever.map() | get_unique_union
docs = retriever_chain.invoke({"question":question})

In [192]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

final_rag_chain = ( {'context': retriever_chain,
                     'question': itemgetter('question')}
                   |prompt
                   |llm|StrOutputParser()
)

final_rag_chain.invoke({'question':question})

'Task decomposition for LLM agents involves breaking down complex tasks into smaller and simpler steps using techniques such as Chain of Thought (CoT) or Tree of Thoughts. It can be done through simple prompting, task-specific instructions, or with the help of an external classical planner in the LLM+P approach.'